In [7]:
import open3d as o3d
import numpy as np
from stl import mesh

# === Load STL ===
your_mesh = mesh.Mesh.from_file(r'workpiece\first_object.stl')
triangles = your_mesh.vectors
facet_normals = your_mesh.normals

# === Build Open3D mesh ===
vertices = triangles.reshape(-1, 3)
tri_indices = np.arange(len(vertices)).reshape(-1, 3)

o3d_mesh = o3d.geometry.TriangleMesh()
o3d_mesh.vertices = o3d.utility.Vector3dVector(vertices)
o3d_mesh.triangles = o3d.utility.Vector3iVector(tri_indices)
o3d_mesh.paint_uniform_color([0.7, 0.7, 0.7])

# === Create wireframe (connected facet edges) ===
wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(o3d_mesh)
wireframe.paint_uniform_color([0, 0, 0])  # black edges

# === Triangle centers ===
centers = triangles.mean(axis=1)

# === Normal lines ===
scale = 2
norms = np.linalg.norm(facet_normals, axis=1, keepdims=True)
facet_normals_unit = facet_normals / norms
normal_endpoints = centers + facet_normals_unit * scale

points = np.vstack((centers, normal_endpoints))
lines = [[i, i + len(centers)] for i in range(len(centers))]

normal_lines = o3d.geometry.LineSet()
normal_lines.points = o3d.utility.Vector3dVector(points)
normal_lines.lines = o3d.utility.Vector2iVector(lines)
normal_lines.paint_uniform_color([0, 0, 1])

# === Vertex points ===
pcd_vertices = o3d.geometry.PointCloud()
pcd_vertices.points = o3d.utility.Vector3dVector(vertices)
pcd_vertices.paint_uniform_color([1, 0, 0])

# === Visualize ===
o3d.visualization.draw_geometries([
    o3d_mesh,
    wireframe,
    pcd_vertices,
    normal_lines
])

In [30]:
import numpy as np

lengths = np.linalg.norm(facet_normals, axis=1)
print("Min length:", lengths.min())
print("Max length:", lengths.max())
print("Mean length:", lengths.mean())

Min length: 6.564726e-08
Max length: 0.00019543787
Mean length: 7.545024e-06
